In [1]:
import keras

Using TensorFlow backend.


In [2]:
import property_inference_interface
interface = property_inference_interface.PropertyInferenceInterface()

meta_params = {
    'first_class': 5, 
    'second_class': 7, 
    'size_of_train_set': 500, 
    'size_of_test_set': 100,
    'subsample_ratio': 1, 
    'flatten': False, 
    
    'is_ran_per_included': False, 
    'ran_dist': 'uniform', 
    'uni_range': 0.2, 
    'normal_std': 0.1,
    
    'model_type': 'CNN'
}

interface.set_meta_params(meta_params)
interface.print_meta_params()

first_class          : 5
second_class         : 7
size_of_train_set    : 500
size_of_test_set     : 100
subsample_ratio      : 1
flatten              : False
is_ran_per_included  : False
ran_dist             : uniform
uni_range            : 0.2
normal_std           : 0.1
model_type           : CNN


In [3]:
interface.prepare_dataset()
interface.print_dataset_shape()

Train dataset
(500, 1, 28, 28) (500,)
Test dataset
(100, 1, 28, 28) (100,)


In [4]:
interface.generate_model(20)
interface.eval_model()

Model (train) accurancy: 1.0


1.0

In [5]:
print('Display the architectur')
interface.model

Display the architectur


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [6]:
interface.generate_set_of_preconditions()
interface.print_set_of_preconditions()
num_first_F, num_second_F, num_unqi_fist_F, num_unqi_secon_F = interface.info_about_set_of_preconditions()

RuntimeError: shape '[-1, 576]' is invalid for input of size 288

In [ ]:
benign_detect_ratio, adversarial_detect_ratio = interface.evaluate_algorithm_on_test_set()

In [ ]:
exp_info = [num_first_F, num_second_F, num_unqi_fist_F, num_unqi_secon_F, benign_detect_ratio, adversarial_detect_ratio]
print(exp_info)

In [ ]:
def single_exp():
    interface.generate_model()
    acc = interface.eval_model()
    if not (acc == 1.0):
        interface.generate_model()
        acc = interface.eval_model()
        
    interface.generate_set_of_preconditions()
    num_first_F, num_second_F, num_unqi_fist_F, num_unqi_secon_F = interface.info_about_set_of_preconditions()
    benign_detect_ratio, adversarial_detect_ratio = interface.evaluate_algorithm_on_test_set(verbose=True)
    exp_info = [num_first_F, num_second_F, num_unqi_fist_F, num_unqi_secon_F, benign_detect_ratio, adversarial_detect_ratio]
    return exp_info    

def exp(num_of_trails, file_name):
    cumulative_exp_info = []
    
    import csv 

    for i in range(num_of_trails):
        print('trail', (i+1))
        single_exp_info = single_exp()
        cumulative_exp_info.append(single_exp_info)

        with open(file_name, mode='a') as file:
            writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(single_exp_info)
        file.close()

        print(single_exp_info)
        
    return cumulative_exp_info

file_name = 'test.csv'
cumulative_exp_info = exp(10, file_name)